In [4]:
# Non-widget row selection → path_key → mastertable slice → QC build

import re
from typing import Iterable, Union, List, Tuple
import pandas as pd

def _parse_row_selection(selection: Union[str, int, Iterable[int], None], n_rows: int) -> List[int]:
    """
    Accepts selections like:
      - None → all rows
      - 7 → single row
      - [1,4,9] or (1,4,9) → explicit list
      - '3' → single row
      - '2,5,7' → list
      - '5-9' or '5:9' → inclusive range
      - '12:' → from 12 to end
      - '1,4-6,10:,0' → mixed
    Interprets indices as positional (iloc), 0-based.
    """
    if selection is None:
        return list(range(n_rows))
    if isinstance(selection, int):
        return [selection] if 0 <= selection < n_rows else []
    if isinstance(selection, (list, tuple, set)):
        return sorted({i for i in map(int, selection) if 0 <= i < n_rows})
    if isinstance(selection, str):
        tokens = [t for t in re.split(r'[,\s]+', selection.strip()) if t]
        out = set()
        for t in tokens:
            if re.fullmatch(r'\d+:', t):  # "a:"
                a = int(t[:-1])
                if a < n_rows:
                    out.update(range(max(a, 0), n_rows))
                continue
            if re.fullmatch(r'\d+[:\-]\d+', t):  # "a:b" or "a-b"
                a, b = re.split(r'[:\-]', t, 1)
                a, b = int(a), int(b)
                if a > b:
                    a, b = b, a
                a = max(a, 0)
                b = min(b, n_rows - 1)
                if a <= b:
                    out.update(range(a, b + 1))
                continue
            if re.fullmatch(r'\d+', t):  # single
                i = int(t)
                if 0 <= i < n_rows:
                    out.add(i)
                continue
            # ignore malformed tokens
        return sorted(out)
    raise TypeError("selection must be None, int, iterable of int, or str")

def slice_mastertable_by_df_pivot(
    df_pivot: pd.DataFrame,
    mastertable: pd.DataFrame,
    selection: Union[str, int, Iterable[int], None],
    path_key_col: str = 'path_key',
    master_filepath_col: str = 'filepath_csv',
    suffix: str = '.csv'
) -> Tuple[pd.DataFrame, List[str]]:
    """
    Select df_pivot rows by iloc-style selection expression, map to path_key+suffix,
    and filter mastertable[master_filepath_col] accordingly.
    Returns (mastertable_slice, selected_files).
    """
    # use positional indexing regardless of df_pivot.index labels
    dfp = df_pivot.reset_index(drop=True)
    idxs = _parse_row_selection(selection, len(dfp))
    if not idxs:
        return mastertable.iloc[0:0].copy(), []

    keys = dfp.loc[idxs, path_key_col].astype(str).tolist()
    files = [f"{k}{suffix}" for k in keys]

    # try exact match, then fallback to endswith for fully-qualified paths
    mcol = mastertable[master_filepath_col].astype(str)
    mask_equal = mcol.isin(files)
    if mask_equal.any():
        mt_slice = mastertable[mask_equal].copy()
    else:
        mask_end = mcol.str.endswith(tuple(files))
        mt_slice = mastertable[mask_end].copy()

    return mt_slice, files

# --- usage ---
# 1) Inspect df_pivot with positional row numbers
display(df_pivot.reset_index(drop=True)[['experiment','analysis','path_key']])  # optional

# 2) Choose rows by expression (examples): '0', '0,2,5', '3-7', '10:'
#    Example: select a single row 0
mastertable_slice, selected_files = slice_mastertable_by_df_pivot(df_pivot, mastertable, selection='0')

# 3) Build QC from the slice (requires your existing build_qc_df)
qc_df_auto = build_qc_df(mastertable_slice, sample_id=['NFL-QC-L','NFL-QC-M','NFL-QC-H'], stdev_range=2)

# 4) Optional: verify
print(selected_files)
display(mastertable_slice.head())
display(qc_df_auto)


In [ ]:
# --- usage ---
# 1) Inspect df_pivot with positional row numbers
display(df_pivot.reset_index(drop=True)[['experiment','analysis','path_key']])  # optional

# 2) Choose rows by expression (examples): '0', '0,2,5', '3-7', '10:'
#    Example: select a single row 0
mastertable_slice, selected_files = slice_mastertable_by_df_pivot(df_pivot, mastertable, selection='0')

# 3) Build QC from the slice (requires your existing build_qc_df)
qc_df_auto = build_qc_df(mastertable_slice, sample_id=['NFL-QC-L','NFL-QC-M','NFL-QC-H'], stdev_range=2)

# 4) Optional: verify
print(selected_files)
display(mastertable_slice.head())
display(qc_df_auto)
